# Computing LCH with Sage

To use this notebook, execute the next block defining the differential operator function. Next, adapt the data in the following example blocks to your cases of interest.

In [7]:
import sympy as sp
from sage.all import *

def alg_mul(*args):
    result = args[0]
    for arg in args[1:]:
        result = result._mul_(arg)
    return result

def differential_operator(expr, generator_images, generator_degrees, algebra=None):
    if expr == 0:
        return 0
    if algebra:
        if len(expr.terms())>1:
            sum  = 0
            for term in expr.terms():
                sum += differential_operator(term, generator_images, generator_degrees, algebra=algebra)
            return sum
        elif len(str(expr.support()[0]).split('*'))>1:
 
            path = str(expr.support()[0]).split('*')
 
            coeff = expr.coefficients()[0]
            algebra_path = []
            for vertex in path:
                for gen in algebra.gens():
                    if vertex in str(gen):
                        algebra_path.append(gen)
            # Apply the derivation relation: d(a * b) = d(a) * b + a * d(b)
   
                        
            first_term = coeff*differential_operator(algebra_path[0], generator_images, generator_degrees, algebra=algebra) * alg_mul(*algebra_path[1:])
            degree_of_first = generator_degrees.get(algebra_path[0], 0)

            second_term = (-1)**degree_of_first *coeff* algebra_path[0] * differential_operator(alg_mul(*algebra_path[1:]), generator_images, generator_degrees, algebra=algebra)
            return first_term + second_term
        else:
            coeff = expr.coefficients()[0]
            if 1/coeff * expr in generator_images:
                return coeff * generator_images[1/coeff * expr]
            else:
                return 0
    else:
        expr = expr.expand()
        if expr in generator_images:
            return generator_images[expr]
        elif expr.is_Pow:
            base, exp = expr.as_base_exp()
            if base in generator_degrees:
                # Apply the chain rule: d(a^n) = n * a^(n-1) * d(a)
                return exp * base**(exp - 1) * differential_operator(base, generator_images, generator_degrees)
            else:
                return 0
        elif expr.is_Add:
            return sp.Add(*[differential_operator(arg, generator_images, generator_degrees) for arg in expr.args])
        elif expr.is_Mul:
            # Apply the derivation relation: d(a * b) = d(a) * b + a * d(b)
            coeff, rest = expr.as_coeff_mul()
            first_term = differential_operator(rest[0], generator_images, generator_degrees) * sp.Mul(*rest[1:]) * coeff
            degree_of_first = generator_degrees.get(rest[0], 0)
            second_term = (-1)**degree_of_first * rest[0] * differential_operator(sp.Mul(*rest[1:]), generator_images, generator_degrees) * coeff
            return first_term + second_term
        else:
            return 0



In [2]:
# This is for P5

P = DiGraph({Integer(0): {Integer(0): 'a', Integer(2): 'y'}, Integer(1): {Integer(1): 'b', Integer(2): 'w'}, Integer(2): {Integer(2): 'c', Integer(0): 'x', Integer(1): 'z'}})
# Define your quiver (documentation at https://doc.sagemath.org/html/en/reference/graphs/sage/graphs/digraph.html)


P_group = P.path_semigroup()
A = P_group.algebra(CC)

e_0, e_1, e_2, a,b,c,w,x,y,z = A.gens()
# Give a name to the generators of the algebra. Preferably, use the same names as the edges of the quiver.
# Run A.gens() in a cell to see the order and names of the edges corresponding to the generators.
# Todo: make this automatic


# Define here how the differential should act on generators.
d_vals = {
    e_0: 0,
    e_1: 0,
    e_2: 0, 
    a: -x*y,
    b: -z*w,
    c: -w*z*x*y - w*z - y*x,
    x: 0,
    y: 0,
    z: 0,
    w: 0
}


# Define here the degrees of the generators.
degrees = {
    e_0: 0,
    e_1: 0,
    e_2: 0,
    a: 1,
    b: 1,
    c: 1,
    x: 0,
    y: 0,
    z: 0,
    w: 0
}

def d(expr):
    return differential_operator(expr, d_vals, degrees, algebra = A)

# After running this cell, you can use the function d to compute the differential of any element of the algebra.
# For example, to compute the differential of a*b, run d(a*b).

In [25]:
d(x*y*z*a*b*a*b*q*s*p*)

y*x + w*z

In [11]:
X = x*y
Y = z*w
Z = x+y+z+w

print(X*Y - Y*X)
print(X*Z-Z*X)
print(Y*Z-Z*Y)

print(X**10)

z*w*x*y + x*y*z*w
y*x*y + w*x*y + x*y*x + x*y*z
y*z*w + w*z*w + z*w*x + z*w*z
x*y*x*y*x*y*x*y*x*y*x*y*x*y*x*y*x*y*x*y


In [18]:
print(d(a*a))

0


In [43]:
expr = d(x*a)

print(expr)


0


In [4]:
# For P1 according to CM

P = DiGraph({Integer(0): {Integer(0): ['a','b','x','y','z']}})

P_group = P.path_semigroup()
A = P_group.algebra(CC)

e_0, a,b,x,y,z = A.gens()

# a,b = sp.symbols('a, b', commutative=False) # degree 1
# x,y,z = sp.symbols('x, y, z', commutative=False) # degree 0

d_vals = {
    e_0: 0,
    a: 1+x+z+x*y*z,
    b: 1-x-z-z*y*x,
    x: 0,
    y: 0,
    z: 0
    }

degrees = {
    e_0: 0,
    a: 1,
    b: 1,
    x: 0,
    y: 0,
    z: 0
}

def d(expr):
    return differential_operator(expr, d_vals, degrees, algebra=A)





In [5]:
x_y = a*(1+y*x)- (1+x*y)*b
x_z = z*a-b*z
y_z = (1+z*y)*a - b*(1+y*z)

print(d(x_y))
print(d(x_z))
print(d(y_z))



2.00000000000000*z + 2.00000000000000*x - 1.00000000000000*x*y + 1.00000000000000*y*x + 2.00000000000000*x*y*z + 2.00000000000000*z*y*x + 2.00000000000000*x*y*x + 2.00000000000000*x*y*z*y*x
2.00000000000000*z*z + 1.00000000000000*x*z + 1.00000000000000*z*x + 1.00000000000000*z*x*y*z + 1.00000000000000*z*y*x*z
2.00000000000000*z + 2.00000000000000*x - 1.00000000000000*y*z + 1.00000000000000*z*y + 2.00000000000000*z*y*z + 2.00000000000000*x*y*z + 2.00000000000000*z*y*x + 2.00000000000000*z*y*x*y*z


## P2

In [2]:
# P = DiGraph({Integer(0): {Integer(0): ['a','b','x','y','z'], Integer(1): ['r','s']}, Integer(1): {Integer(0): ['p','q'], Integer(1): ['c']}})
P = DiGraph({Integer(0): {Integer(0): ['a','b','x','y','z'], Integer(1): ['p','q']}, Integer(1): {Integer(0): ['r','s'], Integer(1): ['c']}})
Q = Quiver(P)
# Define your quiver (documentation at https://doc.sagemath.org/html/en/reference/graphs/sage/graphs/digraph.html)


P_group = P.path_semigroup()
A = P_group.algebra(CC)

e_0, e_1, a,b,c,p,q,r,s,x,y,z = A.gens()
# Give a name to the generators of the algebra. Preferably, use the same names as the edges of the quiver.
# Run A.gens() in a cell to see the order and names of the edges corresponding to the generators.
# Todo: make this automatic


# Define here how the differential should act on generators.
d_vals = {
    e_0: 0,
    e_1: 0,
    a: -e_0 + x + z + x*y*z+s*p*x,
    b: e_0 +x + z+ z*y*x + r*q*x,
    c: 0,
    p: 0,
    q: -p-p*x*y,
    r: 0,
    s: r+x*y*r,
    x: 0,
    y: 0,
    z: 0
}


# Define here the degrees of the generators.
degrees = {
    e_0: 0,
    e_1: 0,
    a: 1,
    b: 1,
    c: 1,
    p: 0,
    q: 1,
    r: -1,
    s: 0,
    x: 0,
    y: 0,
    z: 0
}

def d(expr):
    return differential_operator(expr, d_vals, degrees, algebra = A)

# After running this cell, you can use the function d to compute the differential of any element of the algebra.

# For example, to compute the differential of a*b, run d(a*b).

### GAP Computations

In [4]:
gap.load_package("qpa")
Q = gap.eval("{Integer(0): {Integer(0): ['a','b','x','y','z'], Integer(1): ['p','q']}, Integer(1): {Integer(0): ['r','s'], Integer(1): ['c']} }")
gap.eval("2, [[1,1,1,1,1]]")

TypeError: Gap produced error output
Error, Variable: 'Looped' must have a value
Syntax error: ; expected
\$sage1:=Looped multi-digraph on 2 vertices;;
                ^^^^^
Error, Variable: 'digraph' must have a value

   executing \$sage1:=Looped multi-digraph on 2 vertices;;

In [4]:
# Run this in Z_2
print(d((e_0+z*y)*a+b*(e_0+y*z)))
print(d(a*(e_0+y*x)+(e_0+x*y)*b))
print(d(z*a-b*z))




2.00000000000000*z + 2.00000000000000*x + 1.00000000000000*y*z - 1.00000000000000*z*y + 2.00000000000000*z*y*z + 2.00000000000000*x*y*z + 2.00000000000000*z*y*x + 1.00000000000000*r*q*x + 1.00000000000000*s*p*x + 2.00000000000000*z*y*x*y*z + 1.00000000000000*r*q*x*y*z + 1.00000000000000*z*y*s*p*x
2.00000000000000*z + 2.00000000000000*x + 1.00000000000000*x*y - 1.00000000000000*y*x + 2.00000000000000*x*y*z + 2.00000000000000*z*y*x + 2.00000000000000*x*y*x + 1.00000000000000*r*q*x + 1.00000000000000*s*p*x + 2.00000000000000*x*y*z*y*x + 1.00000000000000*s*p*x*y*x + 1.00000000000000*x*y*r*q*x
-2.00000000000000*z - 1.00000000000000*x*z + 1.00000000000000*z*x + 1.00000000000000*z*x*y*z - 1.00000000000000*z*y*x*z - 1.00000000000000*r*q*x*z + 1.00000000000000*z*s*p*x


# $\implies$ The degree zero homology is commutative in $\Z_2$.

In [7]:
print(d(a*b))

-1.00000000000000*b - 1.00000000000000*a - 1.00000000000000*a*z - 1.00000000000000*a*x + 1.00000000000000*z*b + 1.00000000000000*x*b - 1.00000000000000*a*z*y*x - 1.00000000000000*a*r*q*x + 1.00000000000000*x*y*z*b + 1.00000000000000*s*p*x*b
